In [49]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from tabulate import tabulate
from fredapi import Fred

class StockAnalysis:
    def __init__(self, tickers, start_date, end_date):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.results = []
        self.closing_prices_df = pd.DataFrame()
        self.economic_indicators = {
            'HOUST': "Housing Starts",
        'UNRATE': "Unemployment Rate",
        'CPIAUCSL': "Consumer Price Index for All Urban Consumers: All Items",
        'GDPC1': "Real Gross Domestic Product (Quarterly)",
        'FEDFUNDS': "Effective Federal Funds Rate",
        'PCE': "Personal Consumption Expenditures",
        'INDPRO': "Industrial Production Index",
        'DGS10': "10-Year Treasury Constant Maturity Rate",
        'M2': "M2 Money Stock",
        'GDP': "Gross Domestic Product",
        'DGORDER': "Durable Goods Orders",
        'UMCSENT': "Consumer Sentiment Index",
        'RSXFS': "Retail Sales: Total (excl. Motor Vehicles and Parts Dealers)",
        'BUSINV': "Total Business Inventories",
        'WTISPLC': "West Texas Intermediate (WTI) Crude Oil Spot Price",
        'PCOPPUSDM': "Copper Price",
        'DGS10': 'Interest Rates',
        'PCU325412325412': "Producer Price Index: Pharma",
        'PCU332510332510' : "Producer Price Index: Hardware",
        }

        self.today = datetime.now().strftime('%Y-%m-%d')

    def fetch_stock_data(self, ticker):
        return yf.download(ticker, start=self.start_date, end=self.end_date, progress=False)

    def calculate_slope(self, stock_data):
        x = np.arange(len(stock_data)).reshape(-1, 1)
        y = stock_data["Close"].values.reshape(-1, 1)
        model = LinearRegression()
        model.fit(x, y)
        slope = model.coef_[0][0]
        return slope

    def fetch_fundamental_data(self, ticker):
        fundamental_data = {}
        stock_info = yf.Ticker(ticker)
        try:
            pe_ratio = stock_info.info['trailingPE']
            beta = stock_info.info['beta']
            pb_ratio = stock_info.info['priceToBook']
        except KeyError:
            pe_ratio = None
            beta = None
            pb_ratio = None
        fundamental_data['P/E Ratio'] = pe_ratio
        fundamental_data['Beta'] = beta
        fundamental_data['P/B Ratio'] = pb_ratio
        return fundamental_data

    def fetch_dividend_data(self, ticker):
        dividend_data = []
        stock_info = yf.Ticker(ticker)
        dividends = stock_info.dividends.reset_index()
        for _, row in dividends.iterrows():
            dividend_data.append((row['Date'], row['Dividends']))
        return dividend_data

    def assign_points(self, value, conditions):
        for threshold, points in conditions.items():
            if value < threshold:
                return points
        return 0

    def analyze_tickers(self):
        slope_conditions = {0.10: 0, 0.20: 1, 0.30: 2, 0.40: 3}
        pe_conditions = {7: 0, 14: 4, 15: 3, 25: 2, 35: 1, 40: 0}
        beta_conditions = {0.80: 4, 1.0: 3, 1.4: 2, 1.9: 1, 2.0: 0, 3.0: -1}
        pb_conditions = {1: 2, 2: 1}

        for ticker in self.tickers:
            stock_data = self.fetch_stock_data(ticker)
            self.closing_prices_df[ticker] = stock_data["Close"]
            slope = self.calculate_slope(stock_data)
            fundamental_data = self.fetch_fundamental_data(ticker)
            pe_ratio = fundamental_data['P/E Ratio']
            beta = fundamental_data['Beta']
            pb_ratio = fundamental_data['P/B Ratio']

            slope_points = self.assign_points(slope, slope_conditions)
            pe_points = self.assign_points(pe_ratio, pe_conditions)
            beta_points = self.assign_points(beta, beta_conditions)

            if pb_ratio < 1:
                pb_points = 3
            elif pb_ratio == 1:
                pb_points = 2
            else:
                pb_points = 0

            dividend_data = self.fetch_dividend_data(ticker)
            dividend_points = 1 if len(dividend_data) > 0 else 0

            total_points = slope_points + pe_points + beta_points + pb_points + dividend_points
            combined_points = total_points

            if combined_points < 3:
                recommendation = "Sell"
            elif combined_points < 5:
                recommendation = "Don't buy"
            elif combined_points < 7:
                recommendation = "Buy if nothing else"
            else:
                recommendation = "Strong buy (invest)"

            self.results.append((ticker, slope, slope_points, pe_ratio, pe_points, beta, beta_points, pb_ratio, pb_points, dividend_points, total_points, combined_points, recommendation))

    def fetch_economic_data(self, ticker):
        API_KEY = "69fd7790faf0151554331d3354cd8ab4"
        fred_client = Fred(api_key=API_KEY)
        ticker_data = {}
        for series_id, series_name in self.economic_indicators.items():
            try:
                data = fred_client.get_series(series_id, start_date=self.start_date, end_date=self.today)
                ticker_data[series_name] = data
            except Exception as e:
                print(f"Error fetching data for {series_name} and {ticker}: {e}")
        ticker_df = pd.DataFrame(ticker_data)
        return ticker_df

    def calculate_correlations(self):
        for ticker in self.tickers:
            stock_data = self.fetch_stock_data(ticker)
            economic_data = self.fetch_economic_data(ticker)

            correlation_matrix = self.calculate_correlation_matrix(economic_data, stock_data, ticker)

            # Get the correlation with the stock's closing price and sort
            stock_correlations = correlation_matrix[ticker + ' Close Price'].sort_values(ascending=False)

            # Print the top 5 most correlated economic indicators
            print(f"Top 5 most correlated economic indicators with {ticker} stock:")
            print(stock_correlations.head(6))



    def calculate_correlation_matrix(self, ticker_df, stock_data, ticker):
        ticker_df[ticker + ' Close Price'] = stock_data['Close']
        ticker_position = ticker_df.columns.to_list().index(ticker + ' Close Price')
        columns_reordered = [ticker + ' Close Price'] + ticker_df.columns.to_list()[:ticker_position] + ticker_df.columns.to_list()[ticker_position+1:]
        correlation_matrix = ticker_df[columns_reordered].corr()
        correlation_matrix = correlation_matrix.T
        return correlation_matrix

    def print_results_table(self):
        headers = ["Ticker", "Slope", "Slope Points", "P/E Ratio", "P/E Points", "Beta", "Beta Points", "P/B Ratio", "P/B Points", "Dividend Points", "Total Points", "Combined Points", "Recommendation"]
        table = []
        for result in self.results:
            ticker, slope, slope_points, pe_ratio, pe_points, beta, beta_points, pb_ratio, pb_points, dividend_points, total_points, combined_points, recommendation = result
            table.append([
                ticker,
                f"{slope:.2f}",
                slope_points,
                f"{pe_ratio:.2f}" if pe_ratio is not None else "-",
                pe_points,
                f"{beta:.2f}" if beta is not None else "-",
                beta_points,
                f"{pb_ratio:.2f}" if pb_ratio is not None else "-",
                pb_points,
                dividend_points,
                total_points,
                combined_points,
                recommendation
            ])
        print(tabulate(table, headers=headers))


if __name__ == "__main__":
    tickers_input = input("Enter the tickers (comma-separated): ")
    tickers = [ticker.strip().upper() for ticker in tickers_input.split(",")]
    start_date = (datetime.now() - timedelta(days=6 * 365)).strftime('%Y-%m-%d')
    today = datetime.now().strftime('%Y-%m-%d')

    stock_analysis = StockAnalysis(tickers, start_date, end_date=today)
    stock_analysis.analyze_tickers()
    stock_analysis.calculate_correlations()
    stock_analysis.print_results_table()


Top 5 most correlated economic indicators with MSFT stock:
MSFT Close Price                                                1.000000
M2 Money Stock                                                  0.960343
Retail Sales: Total (excl. Motor Vehicles and Parts Dealers)    0.899679
Producer Price Index: Hardware                                  0.851979
Gross Domestic Product                                          0.847458
Personal Consumption Expenditures                               0.829258
Name: MSFT Close Price, dtype: float64
Ticker      Slope    Slope Points    P/E Ratio    P/E Points    Beta    Beta Points    P/B Ratio    P/B Points    Dividend Points    Total Points    Combined Points  Recommendation
--------  -------  --------------  -----------  ------------  ------  -------------  -----------  ------------  -----------------  --------------  -----------------  -------------------
MSFT         0.17               1        34.93             1     0.9              3        11.62 

In [48]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from tabulate import tabulate
from fredapi import Fred

class StockAnalysis:
    def __init__(self, tickers, start_date, end_date):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.results = []
        self.closing_prices_df = pd.DataFrame()
        self.economic_indicators = {
        'HOUST': "Housing Starts",
        'UNRATE': "Unemployment Rate",
        'CPIAUCSL': "Consumer Price Index for All Urban Consumers: All Items",
        'GDPC1': "Real Gross Domestic Product (Quarterly)",
        'FEDFUNDS': "Effective Federal Funds Rate",
        'PCE': "Personal Consumption Expenditures",
        'INDPRO': "Industrial Production Index",
        'DGS10': "10-Year Treasury Constant Maturity Rate",
        'M2': "M2 Money Stock",
        'GDP': "Gross Domestic Product",
        'DGORDER': "Durable Goods Orders",
        'UMCSENT': "Consumer Sentiment Index",
        'RSXFS': "Retail Sales: Total (excl. Motor Vehicles and Parts Dealers)",
        'BUSINV': "Total Business Inventories",
        'WTISPLC': "West Texas Intermediate (WTI) Crude Oil Spot Price",
        'PCOPPUSDM': "Copper Price",
        'DGS10': 'Interest Rates',
        'PCU325412325412': "Producer Price Index: Pharma",
        'PCU332510332510' : "Producer Price Index: Hardware"
        }
        self.today = datetime.now().strftime('%Y-%m-%d')

    def fetch_stock_data(self, ticker):
        return yf.download(ticker, start=self.start_date, end=self.end_date, progress=False)

    def calculate_slope(self, data):
        x = np.arange(len(data)).reshape(-1, 1)
        y = data.values.flatten()
        valid_indices = ~np.isnan(y)
        x_valid = x[valid_indices]
        y_valid = y[valid_indices]
        
        model = LinearRegression()
        model.fit(x_valid, y_valid)
        slope = model.coef_[0]
        return slope

    def fetch_economic_data(self, ticker):
        API_KEY = "69fd7790faf0151554331d3354cd8ab4"
        fred_client = Fred(api_key=API_KEY)
        ticker_data = {}
        for series_id, series_name in self.economic_indicators.items():
            try:
                data = fred_client.get_series(series_id, start_date=self.start_date, end_date=self.today)
                ticker_data[series_name] = data
            except Exception as e:
                print(f"Error fetching data for {series_name} and {ticker}: {e}")
        ticker_df = pd.DataFrame(ticker_data)
        return ticker_df

    def calculate_correlations(self):
        for ticker in self.tickers:
            stock_data = self.fetch_stock_data(ticker)
            economic_data = self.fetch_economic_data(ticker)

            aligned_data = economic_data.reindex(stock_data.index)
            correlation_matrix = self.calculate_correlation_matrix(aligned_data, stock_data, ticker)
            stock_correlations = correlation_matrix[ticker + ' Close Price'].sort_values(ascending=False)
            
            top_correlations = stock_correlations.head(7)
            
            slopes = []
            for column in top_correlations.index:
                slope = self.calculate_slope(aligned_data[[column]])
                slopes.append(slope)
            
            slopes_df = pd.DataFrame({'Indicator': top_correlations.index, 'Correlation': top_correlations.values, 'Slope': slopes})
            
            print(f"Top 5 most correlated economic indicators with {ticker} stock:")
            print(slopes_df)

    def calculate_correlation_matrix(self, ticker_df, stock_data, ticker):
        ticker_df[ticker + ' Close Price'] = stock_data['Close']
        ticker_position = ticker_df.columns.to_list().index(ticker + ' Close Price')
        columns_reordered = [ticker + ' Close Price'] + ticker_df.columns.to_list()[:ticker_position] + ticker_df.columns.to_list()[ticker_position+1:]
        correlation_matrix = ticker_df[columns_reordered].corr()
        correlation_matrix = correlation_matrix.T
        return correlation_matrix

if __name__ == "__main__":
    tickers_input = input("Enter the tickers (comma-separated): ")
    tickers = [ticker.strip().upper() for ticker in tickers_input.split(",")]
    start_date = (datetime.now() - timedelta(days=6 * 365)).strftime('%Y-%m-%d')
    today = datetime.now().strftime('%Y-%m-%d')

    stock_analysis = StockAnalysis(tickers, start_date, end_date=today)
    stock_analysis.calculate_correlations()


Top 5 most correlated economic indicators with MSFT stock:
                                           Indicator  Correlation       Slope
0                                   MSFT Close Price     1.000000    0.174715
1                                     M2 Money Stock     0.960343    6.542803
2  Retail Sales: Total (excl. Motor Vehicles and ...     0.899679  146.949490
3                     Producer Price Index: Hardware     0.851979    0.063024
4                             Gross Domestic Product     0.847458    5.295638
5                  Personal Consumption Expenditures     0.829258    3.515492
6  Consumer Price Index for All Urban Consumers: ...     0.828915    0.040878
